# Imports

In [1]:
import os
import json
# import urllib2
# import zipfile
from pprint import pprint

import numpy as np
import tensorflow as tf
import keras as k

In [13]:
! conda install tensorflow==1.14.0


/bin/bash: line 1: conda: command not found


# Load data

In [2]:
data_folder = '/content/'



In [3]:
shape = np.loadtxt(os.path.join(data_folder, 'tensor_shape.txt')).astype(int)
tr_idxs = np.loadtxt(os.path.join(data_folder, 'train_indices.txt')).astype(int)
tr_vals = np.loadtxt(os.path.join(data_folder, 'train_values.txt'))
te_idxs = np.loadtxt(os.path.join(data_folder, 'test_indices.txt')).astype(int)
te_vals = np.loadtxt(os.path.join(data_folder, 'test_values.txt'))

# Define util functions

In [29]:
def mape_keras(y_true, y_pred, threshold=0.1):
    v = k.backend.clip(k.backend.abs(y_true), threshold, None)
    diff = k.backend.abs((y_true - y_pred) / v)
    return 100.0 * k.backend.mean(diff, axis=-1)

def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))

def rmse(y_true, y_pred):
    return np.sqrt(np.mean(np.square(y_pred - y_true)))

def mape(y_true, y_pred, threshold=0.1):
    v = np.clip(np.abs(y_true), threshold, None)
    diff = np.abs((y_true - y_pred) / v)
    return 100.0 * np.mean(diff, axis=-1).mean()

def transform(idxs):
    return [idxs[:, i] for i in range(idxs.shape[1])]

# def set_session(device_count=None, seed=0):
#     gpus = tf.config.list_physical_devices('GPU')
#     for gpu in gpus:
#       config = tf.config.experimental.set_memory_growth(gpu, True)
#     print(gpu)
#     session = tf.compat.v1.Session(config=config)
#     tf.compat.v1.keras.backend.set_session(session)

#     np.random.seed(seed)
#     tf.random.set_seed(seed)
#     return session
def set_session(device_count= None, seed = 0):
    gpu_options = tf.compat.v1.GPUOptions(allow_growth=True)

    if device_count is not None:
        # If device count is specified, set it in config
        config = tf.compat.v1.ConfigProto(
            gpu_options=gpu_options,
            device_count=device_count
        )
    else:
        config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)

    # Create and set TensorFlow session
    session = tf.compat.v1.Session(config=config)
    tf.compat.v1.keras.backend.set_session(session)

    # Set random seeds
    np.random.seed(seed)
    tf.random.set_seed(seed)

    return session

def get_metrics(model, x, y, batch_size=1024):
    yp = model.predict(x, batch_size=batch_size, verbose=1).flatten()
    return {
        "rmse": float(rmse(y, yp)),
        "mape": float(mape(y, yp)),
        "mae": float(mae(y, yp))
    }

# Create a CoSTCo model

In [14]:
def create_costco(shape, rank, nc):
    inputs = [k.Input(shape=(1,), dtype="int32") for i in range(len(shape))]
    embeds = [
        k.layers.Embedding(output_dim=rank, input_dim=shape[i])(inputs[i])
        for i in range(len(shape))
    ]
    x = k.layers.Concatenate(axis=1)(embeds)
    x = k.layers.Reshape(target_shape=(rank, len(shape), 1))(x)
    x = k.layers.Conv2D(
        nc,
        kernel_size=(1, len(shape)),
        activation="relu",
        padding="valid"
    )(x)
    x = k.layers.Conv2D(
        nc,
        kernel_size=(rank, 1),
        activation="relu",
        padding="valid"
    )(x)
    x = k.layers.Flatten()(x)
    x = k.layers.Dense(nc, activation="relu")(x)
    outputs = k.layers.Dense(1, activation="relu")(x)
    model = k.Model(inputs=inputs, outputs=outputs)

    return model

# Set hyper-parameters

In [15]:
lr = 1e-4
rank = 20
nc = rank
epochs = 50
batch_size = 256

seed = 3
verbose = 1

# Train with early stopping

In [30]:
set_session(device_count={"GPU": 0}, seed=seed)

In [ ]:
set_session(device_count={"GPU": 0}, seed=seed)
optim = k.optimizers.Adam(lr=lr)

model = create_costco(shape, rank, nc)
model.compile(optim, loss=["mse"], metrics=["mae", mape_keras])
hists = model.fit(
    x=transform(tr_idxs),
    y=tr_vals,
    verbose=verbose,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[k.callbacks.EarlyStopping(
        monitor="val_mean_absolute_error",
        patience=10,
        restore_best_weights=True)],
);

# Evaluation model prediction over the test set

In [37]:
tr_info = get_metrics(model, transform(tr_idxs), tr_vals)
te_info = get_metrics(model, transform(te_idxs), te_vals)

21/21 [==============================] - 0s 2ms/step


In [25]:
pprint({'train': tr_info})

{'train': {'mae': 0.038172314558212746,
           'mape': 22.542594515942145,
           'rmse': 0.08550530942846674}}


In [26]:
pprint({'test': te_info})

{'test': {'mae': 0.09540652910370986,
          'mape': 56.83021305364622,
          'rmse': 0.19719356878624342}}


In [38]:
pprint({'train': tr_info})

{'train': {'mae': 0.037989188151755934,
           'mape': 22.97265276326988,
           'rmse': 0.08279467166149128}}


In [39]:
pprint({'test': te_info})

{'test': {'mae': 0.09717790047385264,
          'mape': 57.92255575443585,
          'rmse': 0.19743315658205285}}
